In [1]:
"""Script permettant d'explorer les données d'un fichier XML et d'interroger une API"""
################################################################################
# fichier  : td4.py
# Auteur : RAKOTOSON Loic
################################################################################

"Script permettant d'explorer les données d'un fichier XML et d'interroger une API"

In [2]:
################################################################################
# Importation de fonctions externes :
import pandas as pd
import json
from lxml import etree
from requests import get, HTTPError
# Definition locale de fonctions :

In [3]:
def afficheXML(liste):
    for elem in liste:
        if isinstance(elem, etree._Element):
            print(etree.tostring(elem,encoding='unicode'))
        else:
            print(elem)

In [4]:
def afficheTexte(liste):
    for elem in liste:
        if isinstance(elem, etree._Element):
            if elem.text:
                print(elem.text)
            afficheTexte(elem.xpath('./*'))
        else:
            print(elem)

In [5]:
def getJsonData(base, dico):
    with get(base, params = dico) as response:
        return response.json()

In [6]:
def getMatrix(base, dico, metric):
    """
    Retourne la matrice de la métrique
    :base: base url
    :dico: API params
    :metric: distances|durations
    :return: DataFrame
    """
    reponse = getJsonData(base, dico)
    df = pd.DataFrame(
        reponse[metric],
        columns=[
            ",".join(map(str, d['location'])) for d in reponse['destinations']
        ],
        index=[",".join(map(str, d['location'])) for d in reponse['sources']])
    return df

In [7]:
def cleanGPS(GPSlist):
    """
    Renvoie un str formaté pour l'API
    :GPSlist: [lat, lon] du XML
    :return: str
    """
    raw = list(map(str,map(int,[coord.replace('.', '') for coord in GPSlist])))
    if '-' in raw[0]:
        lat = raw[0][:3] + '.' + raw[0][3:10]
    else:
        lat = raw[0][:2] + '.' + raw[0][2:9]
    if '-' in raw[1]:
        lon = raw[1][:2] + '.' + raw[1][2:9]
    else:
        lon = raw[1][0] + '.' + raw[1][1:8]
    return lon + ',' + lat

In [8]:
################################################################################
# Corps principal du programme :

In [9]:
base_url = "https://api.openrouteservice.org/matrix"
APIKEY = '5b3ce3597851110001cf62486cfda2a850de46e4a43253fa6adffd31'

# Exercice 1: découverte de l'API Matrix de openrouteservice
Trouver les coordonnées GPS des villes de Rennes, Paris, Nantes et Brest, et mémorisez-les (dans cet ordre)  au début du programme principal sous forme d'une liste Python de chaînes de caractères représentant chacune une paire de coordonnées séparées par une virgule:

In [10]:
villesGPS = [
    '-1.6800198,48.1113387', '2.3514992,48.8566101',
    '-1.553621,47.218371', '-4.486076,48.390394'
]

Écrire une fonction `getJsonData()` prenant en paramètre l'URL de base de l'API Matrix et un dictionnaire des paramètres de la requête (clé = nom du paramètre; valeur=valeur du paramètre) et retournant les données JSON renvoyées par le service, sous forme d'une structure de données Python.

In [11]:
data = {
    "api_key" : APIKEY,
    "profile": "driving-car",
    "locations": "|".join(villesGPS),
    "sources": "0,1",
    "destinations": "2,3",
    "metrics": "distance|duration",
    "units": "km",
    
}
reponse = getJsonData(base_url, data)
print(reponse)

{'distances': [[106.9, 241.54], [381.01, 590.73]], 'durations': [[5018.62, 9540.34], [14013.42, 21583.63]], 'destinations': [{'location': [-1.55357, 47.218302], 'snapped_distance': 8.61}, {'location': [-4.486227, 48.390272], 'snapped_distance': 17.53}], 'sources': [{'location': [-1.680245, 48.111335], 'snapped_distance': 16.73}, {'location': [2.351055, 48.856717], 'snapped_distance': 34.62}], 'info': {'service': 'matrix', 'engine': {'version': '6.1.0', 'build_date': '2020-03-19T12:54:55Z'}, 'attribution': 'openrouteservice.org | OpenStreetMap contributors', 'timestamp': 1585407905686, 'system_message': '', 'query': {'profile': 'driving-car', 'units': 'km'}}}


Dans le programme principal, écrire le code Python permettant d'obtenir la matrice des distances avec Rennes et Paris comme villes d'origine, Brest et Nantes comme villes de destination, et d'afficher cette matrice ligne par ligne.

In [12]:
matrix = getMatrix(base_url, data, "distances")
matrix

,"-1.55357,47.218302","-4.486227,48.390272"
"-1.680245,48.111335",106.90,241.54
"2.351055,48.856717",381.01,590.73


# Exercice 2: utilisation des données XML des points de vente de carburant et des données en ligne de l'API
Choisir une voiture et déterminer son type de carburant et sa consommation moyenne en allure «route». Mémoriser ces informations dans 2 variables «carburant» et «conso» de votre programme.

In [13]:
carburant = "Gazole"
conso = "4.1"

La première étape du voyage est de faire le plein d'essence sur Rennes au tarif le plus avantageux pour votre carburant.A l'aide des données du fichier *PrixCarburants_quotidien_AAAAMMJJ.xml*, déterminez quel est ce point de vente le plus avantageux pour vous à Rennes. Affichez ce prix et l'adresse complète du point de vente correspondant.  
Remarque: dans le fichier XML, le nom de ville peut apparaître sous la forme «Rennes» ou «RENNES».

In [14]:
root = etree.parse("PrixCarburants_quotidien_20200316.xml").getroot()
gazoleRennes = list(
    map(
        int,
        root.xpath(
            "pdv[contains('Rennes RENNES', ville)]/prix[@nom = '{}']//@valeur".
            format(carburant))))

In [15]:
# Le meilleur à Rennes
bestRennes = root.xpath(
    "pdv[contains('Rennes RENNES', ville)][prix/@nom = '{}' and prix/@valeur = '{}']"
    .format(carburant, min(gazoleRennes)))[0]

In [16]:
# Le pire à Rennes
maxRennes = root.xpath(
    "pdv[contains('Rennes RENNES', ville)][prix/@nom = '{}' and prix/@valeur = '{}']"
    .format(carburant, max(gazoleRennes)))[0]

In [17]:
chaine = "Le meilleur point de vente se trouve à {} au prix de {}€".format(
    *bestRennes.xpath("adresse//text()|prix[@nom = '{}']/@valeur".format(
        carburant)))
print(chaine)

Le meilleur point de vente se trouve à Rue Jules Vallès au prix de 1206€


La deuxième (grande) étape de votre voyage est de vous ravitailler en carburant avant de passer la frontière. Le voyage jusqu'à ce point de vente se fera donc avec le carburant au meilleur tarif que vous avez trouvé sur Rennes. On supposera que la suite du voyage jusqu'à Rome se fera au tarif du carburant chargé au point de ravitaillement.  
Pour ce point de ravitaillement, vous avez quelques contraintes:

- Il devra se trouver dans le département de Savoie (**73**) ou de Haute Savoie (**74**)
- Vous aurez besoin de faire un petit tour aux toilettes avant de repartir (service «Toilettes publiques» obligatoire)
- Vous aurez besoin d'acheter quelque chose à grignoter (service "Boutique alimentaire" ou "Restauration à emporter" obligatoire)
- Vous aurez besoin de retirer de l'argent liquide (service «DAB (Distributeur automatique de billets)» obligatoire)

Déterminez une expression Xpath unique permettant d'obtenir les nœuds «point de vente» répondant à l'ensemble des contraintes ci-dessus. Afficher le nombre de pointde vente correspondants obtenus

In [18]:
request = "pdv[starts-with(@cp, '73') or starts-with(@cp, '74')]"\
"[contains('Toilettes publiques', service)]"\
"[services[service = 'Boutique alimentaire' or service = 'Restauration à emporter']]"\
"[contains('DAB', service)]"

nodes = root.xpath(request)
print(len(nodes))

128


Parmi les points de ventes retournés, déterminez lequel permet d'optimiser le prix global en carburant du voyage, en tenant compte des deux distances parcourues, Rennes → ravitaillement et ravitaillement → Rome (utilisez l'API Matrix pour calculer ces distances), et du tarif du carburant pour ces deux portions.

On regroupe en premier lieu toutes les locations, les sources comprises.  
On enregistre ensuite les paramètres qui seront nécessaires pour effectuer la requête dans un dictionnaire *params*.  
Enfin, on calcule la matrice avec l'API Matrix, en passant par la fonction `getMatrix()`.

In [19]:
Rome = "12.4963655,41.9027835"
Rennes = cleanGPS(bestRennes.xpath("@latitude | @longitude"))
stations = [
    cleanGPS(node.xpath("@latitude | @longitude")) for node in nodes
    if node.xpath("@latitude")[0] != ''
]

params = {
    "api_key":
    APIKEY,
    "profile":
    "driving-car",
    "locations":
    "|".join([Rennes, Rome] + stations),
    "sources":
    "0,1",
    "destinations":
    ",".join(map(str, range(2,
                            len([Rennes, Rome] + stations) - 1))),
    "metrics":
    "distance|duration",
    "units":
    "km",
}

RennesRome = getMatrix(base_url, params, "distances")

RennesRome

,"5.896539,45.593435","5.902134,45.56322","5.886649,45.609687","5.886996,45.609555","5.902134,45.56322","5.908618,45.699637","5.929508,45.674322","5.900294,45.694187","5.943622,45.683149","5.900294,45.694187",...,"6.630034,45.943851","6.632461,45.940459","6.305277,46.06987","6.641215,45.900187","6.143767,45.913756","6.143767,45.913756","6.10714,45.894219","6.10714,45.894219","6.10714,45.894219","6.106811,45.894169"
"-1.71045,48.104349",874.09,877.65,875.41,875.46,877.65,889.74,881.07,883.68,885.62,883.68,...,895.87,896.30,867.36,903.54,866.32,866.32,868.24,868.24,868.24,868.22
"12.496486,41.902874",891.88,893.93,895.40,895.44,893.93,904.79,902.84,903.67,903.22,903.67,...,847.61,843.73,879.75,846.41,917.51,917.51,935.19,935.19,935.19,935.17


On a supposé que la voiture a une autonomie d'environ 900km, et que, partant avec le pleinde carburant de Rennes, nous devrons refaire un ravitaillement juste avant de passer la frontière.  
Symétriquement, la distance entre le point de ravitaillement et Rome ne doit pas éxcéder l'autonomie.  
On supprime donc les colonnes vérifiant cette condition.

In [20]:
RennesRome = RennesRome.loc[:, ~(RennesRome > 900).any()]
RennesRome

,"5.896539,45.593435","5.902134,45.56322","5.886649,45.609687","5.886996,45.609555","5.902134,45.56322","5.89429,45.562299","5.97708,45.525004","5.9532,45.573621","5.88592,45.593559","5.884035,45.592951",...,"6.631579,46.043364","6.564414,46.06095","6.582205,46.108926","6.59047,46.102409","6.277382,46.007926","6.629006,45.952962","6.632461,45.940459","6.630034,45.943851","6.632461,45.940459","6.305277,46.06987"
"-1.71045,48.104349",874.09,877.65,875.41,875.46,877.65,878.37,890.48,880.48,874.0,874.16,...,892.59,880.89,891.92,890.55,869.54,895.01,896.30,895.87,896.30,867.36
"12.496486,41.902874",891.88,893.93,895.40,895.44,893.93,894.64,881.46,890.01,892.8,892.96,...,864.74,858.52,868.15,866.78,889.75,846.75,843.73,847.61,843.73,879.75


Enfin, on supprime toutes les colonnes dont la somme des distances est supérieure à la somme des distance minimale.  
Il ne devra en rester qu'une. Il s'agit de la station la plus otpimisée.

In [21]:
cols = RennesRome.columns[RennesRome.sum(axis=0) > RennesRome.sum(
    axis=0).min()]
best = RennesRome.drop(cols, axis=1)
best

,"6.564414,46.06095"
"-1.71045,48.104349",880.89
"12.496486,41.902874",858.52


Donnez le prix en carburant de ce voyage optimisé, et la différence avec le prix le plus élevé (y compris en faisant le plein le plus cher à Rennes), avec les mêmes contraintes sur le point de ravitaillement.

In [22]:
RennesMax = cleanGPS(maxRennes.xpath("@latitude | @longitude"))
params['locations'] = "|".join([RennesMax, Rome] + stations)
maxcarb = getMatrix(base_url, params, "distances")
maxcarb = maxcarb.loc[:, ~(maxcarb > 900).any()]
maxcarb = maxcarb.drop(
    maxcarb.columns[maxcarb.sum(axis=0) > maxcarb.sum(axis=0).min()], axis=1)

In [23]:
worst = RennesRome[RennesRome.columns[RennesRome.sum(axis=0) == RennesRome.sum(
    axis=0).max()]]

In [24]:
prixbest = float(
    bestRennes.xpath("prix[@nom = '{}']/@valeur".format(carburant))[0][0] +
    '.' +
    bestRennes.xpath("prix[@nom = '{}']/@valeur".format(carburant))[0][1:])

In [25]:
pd.DataFrame({
    'best': best.sum(axis=0).reset_index(drop=True),
    'worst': worst.sum(axis=0).reset_index(drop=True),
    'maxcarb': maxcarb.sum(axis=0).reset_index(drop=True)
}) * prixbest

,best,worst,maxcarb
0,2097.72846,2143.1223,2099.01888


Se ravitailler dans la station la moins avantageuse augmente le coût de 46 euros. En passant par la station la plus chère de Rennes, le coût n'augmente que de 1.30 euro.

Calculer également la durée estimée du voyage optimisé et celle du voyage le plus court en temps (même point de vente à Rennes et mêmes contraintes pour le point deravitaillement)

In [26]:
params['locations'] = "|".join([Rennes, Rome] + stations)
duration = getMatrix(base_url, params, "durations")
duration = duration[duration.columns[duration.sum(axis=0) == duration.sum(
    axis=0).min()].append(best.columns)]
duration.loc['Total'] = duration.sum(axis = 0)
duration

,"6.861136,45.911747","6.564414,46.06095"
"-1.71045,48.104349",30533.77,29142.84
"12.496486,41.902874",28850.62,30716.04
Total,59384.39,59858.88


Il y a environ 8h de différence entre les deux trajets.